In [1]:
# packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [2]:
from ipynb.fs.full.f_dist_gender_helper import calc_gender_dist_emp

In [3]:
def calc_gender_impact(sc,analysis):
    
    """
    Example:
    sc = "Low_RE_Cost_95_by_2035"
    analysis = "v3.5jedi"
    """

    scenario = "StdScen21_"+sc+"_annual_state_"+analysis
    state_name = pd.read_excel(r"../Census/State_Abbr.xlsx")
    JEDI_result = pd.read_csv(r"../ReEDS_to_Jobs/output/JEDI_output_"+scenario+".csv")
    list_of_states = JEDI_result["state"].unique()

    # calculate mid-case, lower bound, upper bound
    for case in ["mid","lb","ub"]:
        for m in list_of_states:
            JEDI_output_state = JEDI_result.query("state==@m")
            JEDI_output_state = JEDI_output_state.set_index('i')       
            JEDI_output_state = JEDI_output_state.drop(columns="state")
            JEDI_output_state = JEDI_output_state.transpose()
            state_full = state_name.query("Abbr==@m").iloc[0,0]
            jobs_sum = calc_gender_dist_emp(JEDI_output_state,state_full,case)

            jobs_sum["state"] = m
            if m == "AL":
                all_jobs = jobs_sum.copy()
            else:
                all_jobs = all_jobs.append(jobs_sum)

        all_jobs["case"] = case
        if case == "mid":
            all_jobs_cases = all_jobs.copy()
        else:
            all_jobs_cases = all_jobs_cases.append(all_jobs)

    # calculate the national impact
    all_jobs_national = all_jobs_cases.copy()
    all_jobs_national = all_jobs_national.reset_index()
    all_jobs_national = all_jobs_national.groupby(["i","case"]).sum()
    all_jobs_national["f_perc"] = all_jobs_national["Female"]/all_jobs_national["Total"]*100
    all_jobs_national["state"] = "US"
    all_jobs_national = all_jobs_national.reset_index().set_index("i")

    # export all raw data
    all_jobs_cases_combined = all_jobs_cases.append(all_jobs_national)
    all_jobs_cases_combined.to_excel(r"output/Dist_gender_"+scenario+".xlsx")

    # export all analysed data 
    for case in ["mid","lb","ub"]:
        all_jobs_female = all_jobs_cases_combined.copy().reset_index()
        all_jobs_female = all_jobs_female.query("case==@case")
        all_jobs_female = all_jobs_female.pivot(index="state", columns = "i", values = "f_perc")
        all_jobs_female["EmpChange_1Y"] = all_jobs_female[2050]/all_jobs_female[2022]*100 - 100
        all_jobs_female["EmpChange_5Y"] = (all_jobs_female[[2046,2047,2048,2049,2050]].mean(axis=1))/all_jobs_female[2022]*100-100
        all_jobs_female["EmpChange_10Y"] = (all_jobs_female[[2041,2042,2043,2044,2045,2046,2047,2048,2049,2050]].mean(axis=1))/all_jobs_female[2022]*100-100
        all_jobs_female["20s"] = all_jobs_female[[2022,2023,2024,2025,2026,2027,2028,2029,2030]].mean(axis=1)
        all_jobs_female["40s"] = all_jobs_female[[2041,2042,2043,2044,2045,2046,2047,2048,2049,2050]].mean(axis=1)
        all_jobs_female["EmpChange_dec"] = all_jobs_female["40s"]/all_jobs_female["20s"]*100 - 100
        all_jobs_female.to_csv(r"output/Dist_female_"+scenario+"_"+case+".csv")

In [4]:
"""
sc = "Low_RE_Cost_95_by_2035"
analysis = "v6.2jedi_mid"
scenario = "StdScen21_"+sc+"_annual_state_"+analysis
state_name = pd.read_excel(r"../Census/State_Abbr.xlsx")
JEDI_result = pd.read_csv(r"../ReEDS_to_Jobs/output/JEDI_output_"+scenario+".csv")
list_of_states = JEDI_result["state"].unique()

    # calculate mid-case, lower bound, upper bound
for case in ["mid","lb","ub"]:
    for m in list_of_states:
        JEDI_output_state = JEDI_result.query("state==@m")
        JEDI_output_state = JEDI_output_state.set_index('i')       
        JEDI_output_state = JEDI_output_state.drop(columns="state")
        JEDI_output_state = JEDI_output_state.transpose()
        state_full = state_name.query("Abbr==@m").iloc[0,0]
        jobs_sum = calc_gender_dist_emp(JEDI_output_state,state_full,case)

        jobs_sum["state"] = m
        if m == "AL":
            all_jobs = jobs_sum.copy()
        else:
            all_jobs = all_jobs.append(jobs_sum)

    all_jobs["case"] = case
    if case == "mid":
        all_jobs_cases = all_jobs.copy()
    else:
        all_jobs_cases = all_jobs_cases.append(all_jobs)

    # calculate the national impact
all_jobs_national = all_jobs_cases.copy()
all_jobs_national = all_jobs_national.reset_index()
all_jobs_national = all_jobs_national.groupby(["i","case"]).sum()
all_jobs_national["f_perc"] = all_jobs_national["Female"]/all_jobs_national["Total"]*100
all_jobs_national["state"] = "US"
all_jobs_national = all_jobs_national.reset_index().set_index("i")

    # export all raw data
all_jobs_cases_combined = all_jobs_cases.append(all_jobs_national)
    #all_jobs_cases_combined.to_excel(r"output/Dist_gender_"+scenario+".xlsx")

    # export all analysed data 
for case in ["mid","lb","ub"]:
    all_jobs_female = all_jobs_cases_combined.copy().reset_index()
    all_jobs_female = all_jobs_female.query("case==@case")
    all_jobs_female = all_jobs_female.pivot(index="state", columns = "i", values = "f_perc")
    all_jobs_female["EmpChange_1Y"] = all_jobs_female[2050]/all_jobs_female[2022]*100 - 100
    all_jobs_female["EmpChange_5Y"] = (all_jobs_female[[2046,2047,2048,2049,2050]].mean(axis=1))/all_jobs_female[2022]*100-100
    all_jobs_female["EmpChange_10Y"] = (all_jobs_female[[2041,2042,2043,2044,2045,2046,2047,2048,2049,2050]].mean(axis=1))/all_jobs_female[2022]*100-100
    all_jobs_female["20s"] = all_jobs_female[[2022,2023,2024,2025,2026,2027,2028,2029,2030]].mean(axis=1)
    all_jobs_female["40s"] = all_jobs_female[[2041,2042,2043,2044,2045,2046,2047,2048,2049,2050]].mean(axis=1)
    all_jobs_female["EmpChange_dec"] = all_jobs_female["40s"]/all_jobs_female["20s"]*100 - 100
"""

'\nsc = "Low_RE_Cost_95_by_2035"\nanalysis = "v6.2jedi_mid"\nscenario = "StdScen21_"+sc+"_annual_state_"+analysis\nstate_name = pd.read_excel(r"../Census/State_Abbr.xlsx")\nJEDI_result = pd.read_csv(r"../ReEDS_to_Jobs/output/JEDI_output_"+scenario+".csv")\nlist_of_states = JEDI_result["state"].unique()\n\n    # calculate mid-case, lower bound, upper bound\nfor case in ["mid","lb","ub"]:\n    for m in list_of_states:\n        JEDI_output_state = JEDI_result.query("state==@m")\n        JEDI_output_state = JEDI_output_state.set_index(\'i\')       \n        JEDI_output_state = JEDI_output_state.drop(columns="state")\n        JEDI_output_state = JEDI_output_state.transpose()\n        state_full = state_name.query("Abbr==@m").iloc[0,0]\n        jobs_sum = calc_gender_dist_emp(JEDI_output_state,state_full,case)\n\n        jobs_sum["state"] = m\n        if m == "AL":\n            all_jobs = jobs_sum.copy()\n        else:\n            all_jobs = all_jobs.append(jobs_sum)\n\n    all_jobs["case"]